## 1. The NIST Special Publication 800-63B
<p>If you – 50 years ago – needed to come up with a secret password you were probably part of a secret espionage organization or (more likely) you were pretending to be a spy when playing as a kid. Today, many of us are forced to come up with new passwords <em>all the time</em> when signing into sites and apps. As a password <em>inventeur</em> it is your responsibility to come up with good, hard-to-crack passwords. But it is also in the interest of sites and apps to make sure that you use good passwords. The problem is that it's really hard to define what makes a good password. However, <em>the National Institute of Standards and Technology</em> (NIST) knows what the second best thing is: To make sure you're at least not using a <em>bad</em> password. </p>
<p>In this notebook, we will go through the rules in <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a> which details what checks a <em>verifier</em> (what the NIST calls a second party responsible for storing and verifying passwords) should perform to make sure users don't pick bad passwords. We will go through the passwords of users from a fictional company and use python to flag the users with bad passwords. But us being able to do this already means the fictional company is breaking one of the rules of 800-63B:</p>
<blockquote>
  <p>Verifiers SHALL store memorized secrets in a form that is resistant to offline attacks. Memorized secrets SHALL be salted and hashed using a suitable one-way key derivation function.</p>
</blockquote>
<p>That is, never save users' passwords in plaintext, always encrypt the passwords! Keeping this in mind for the next time we're building a password management system, let's load in the data.</p>
<p><em>Warning: The list of passwords and the fictional user database both contain <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have not been filtered in any way and include words that are explicit, derogatory and offensive.</em></p>

In [127]:
# Importing the pandas module
# ... YOUR CODE FOR TASK 1 ...
import pandas as pd

# Loading in datasets/users.csv 
users = pd.read_csv('datasets/users.csv')

# Printing out how many users we've got
# ... YOUR CODE FOR TASK 1 ...
print(users)

# Taking a look at the 12 first users
# ... YOUR CODE FOR TASK 1 ...
users.head(12)

      id           user_name          password
0      1      vance.jennings          joobheco
1      2      consuelo.eaton        0869347314
2      3     mitchel.perkins        fabypotter
3      4      odessa.vaughan         aharney88
4      5      araceli.wilder        acecdn3000
5      6    shawn.harrington           5278049
6      7          evelyn.gay            master
7      8         noreen.hale            murphy
8      9         gladys.ward           lwsves2
9     10     brant.zimmerman  1190KAREN5572497
10    11       leanna.abbott          aivlys24
11    12     milford.hubbard           hubbard
12    13           mamie.fox        mitonguito
13    14       jamie.cochran            310356
14    15  nathaniel.robinson         angelmajo
15    16          lorrie.gay           oZ4k0QE
16    17        domingo.dyer           chelsea
17    18      martin.pacheco           zvc1939
18    19       shelby.massey            nickgd
19    20     rosella.barrett        O2gv3LlcfG
20    21     

id         user_name          password
0    1    vance.jennings          joobheco
1    2    consuelo.eaton        0869347314
2    3   mitchel.perkins        fabypotter
3    4    odessa.vaughan         aharney88
4    5    araceli.wilder        acecdn3000
5    6  shawn.harrington           5278049
6    7        evelyn.gay            master
7    8       noreen.hale            murphy
8    9       gladys.ward           lwsves2
9   10   brant.zimmerman  1190KAREN5572497
10  11     leanna.abbott          aivlys24
11  12   milford.hubbard           hubbard

## 2. Passwords should not be too short
<p>If we take a look at the first 12 users above we already see some bad passwords. But let's not get ahead of ourselves and start flagging passwords <em>manually</em>. What is the first thing we should check according to the NIST Special Publication 800-63B?</p>
<blockquote>
  <p>Verifiers SHALL require subscriber-chosen memorized secrets to be at least 8 characters in length.</p>
</blockquote>
<p>Ok, so the passwords of our users shouldn't be too short. Let's start by checking that!</p>

In [129]:
# Calculating the lengths of users' passwords
users['length'] = users['password'].apply(len)

# Flagging the users with too short passwords
users['too_short'] = users.length < 8

# Counting and printing the number of users with too short passwords
# ... YOUR COD['length']E FOR TASK 2 ...
print(users[users.length < 8])

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 2 ...
print(users.head(12))

      id            user_name password  length too_short
5      6     shawn.harrington  5278049       7      True
6      7           evelyn.gay   master       6      True
7      8          noreen.hale   murphy       6      True
8      9          gladys.ward  lwsves2       7      True
11    12      milford.hubbard  hubbard       7      True
13    14        jamie.cochran   310356       6      True
15    16           lorrie.gay  oZ4k0QE       7      True
16    17         domingo.dyer  chelsea       7      True
17    18       martin.pacheco  zvc1939       7      True
18    19        shelby.massey   nickgd       6      True
22    23        jenny.woodard  woodard       7      True
25    26         dianna.munoz    AJ9Da       5      True
26    27         julia.savage   ewokzs       6      True
28    29      joaquin.walters  YyGjz8E       7      True
30    31         rosanna.reid     reid       4      True
34    35         roger.golden  jOYZBs8       7      True
38    39          gus.padilla  

## 3.  Common passwords people use
<p>Already this simple rule flagged a couple of offenders among the first 12 users. Next up in Special Publication 800-63B is the rule that</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets against a list that contains values known to be commonly-used, expected, or compromised.</p>
  <ul>
  <li>Passwords obtained from previous breach corpuses.</li>
  <li>Dictionary words.</li>
  <li>Repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</li>
  <li>Context-specific words, such as the name of the service, the username, and derivatives thereof.</li>
  </ul>
</blockquote>
<p>We're going to check these in order and start with <em>Passwords obtained from previous breach corpuses</em>, that is, websites where hackers have leaked all the users' passwords. As many websites don't follow the NIST guidelines and encrypt passwords there now exist large lists of the most popular passwords. Let's start by loading in the 10,000 most common passwords which I've taken from <a href="https://github.com/danielmiessler/SecLists/tree/master/Passwords">here</a>.</p>

In [131]:
# Reading in the top 10000 passwords
common_passwords = pd.read_csv('datasets/10_million_password_list_top_10000.txt', header=None, squeeze=True)

# Taking a look at the top 20
#  YOUR CODE FOR TASK 3 
print(common_passwords.head())
print(type(common_passwords))
print(common_passwords.head(20))

0       123456
1     password
2     12345678
3       qwerty
4    123456789
Name: 0, dtype: object
<class 'pandas.core.series.Series'>
0        123456
1      password
2      12345678
3        qwerty
4     123456789
5         12345
6          1234
7        111111
8       1234567
9        dragon
10       123123
11     baseball
12       abc123
13     football
14       monkey
15      letmein
16       696969
17       shadow
18       master
19       666666
Name: 0, dtype: object


## 4.  Passwords should not be common passwords
<p>The list of passwords was ordered, with the most common passwords first, and so we shouldn't be surprised to see passwords like <code>123456</code> and <code>qwerty</code> above. As hackers also have access to this list of common passwords, it's important that none of our users use these passwords!</p>
<p>Let's flag all the passwords in our user database that are among the top 10,000 used passwords.</p>

In [133]:
# Flagging the users with passwords that are common passwords
users['common_password'] = users['password'].isin(common_passwords)

# Counting and printing the number of users using common passwords
# ... YOUR CODE FOR TASK 4 ...
common_pass_users = users[users['password'].isin(common_passwords)]
print(len(common_pass_users))

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 4 ...
common_pass_users.head(12)

129


id            user_name   password  length too_short common_password
6      7           evelyn.gay     master       6      True            True
7      8          noreen.hale     murphy       6      True            True
16    17         domingo.dyer    chelsea       7      True            True
21    22      leticia.sanford   cocacola       8     False            True
51    52         ahmad.hopper  123456789       9     False            True
58    59  rosalinda.rodriquez     golden       6      True            True
78    79         roland.mcgee    thx1138       7      True            True
81    82         seymour.ball   midnight       8     False            True
84    85      dorian.caldwell     dorian       6      True            True
89    90        jordan.hurley     hurley       6      True            True
101  102            dee.mccoy     monkey       6      True            True
112  113        juliette.rios   juliette       8     False            True

## 5. Passwords should not be common words
<p>Ay ay ay! It turns out many of our users use common passwords, and of the first 12 users there are already two. However, as most common passwords also tend to be short, they were already flagged as being too short. What is the next thing we should check?</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] dictionary words.</p>
</blockquote>
<p>This follows the same logic as before: It is easy for hackers to check users' passwords against common English words and therefore common English words make bad passwords. Let's check our users' passwords against the top 10,000 English words from <a href="https://github.com/first20hours/google-10000-english">Google's Trillion Word Corpus</a>.</p>

In [135]:
# Reading in a list of the 10000 most common words
words = pd.read_csv('datasets/google-10000-english.txt', header=None, squeeze=True)
print(words.head())

# Flagging the users with passwords that are common words
users['password_lower'] = users['password'].str.lower()
users['common_word'] = users['password_lower'].isin(words)

# Counting and printing the number of users using common words as passwords
# ... YOUR CODE FOR TASK 5 ...
common_word_users = users[users['password_lower'].isin(words)]
print(len(common_word_users))

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 5 ...
common_word_users.head(12)

0    the
1     of
2    and
3     to
4      a
Name: 0, dtype: object
137


id            user_name  password  length too_short common_password  \
6      7           evelyn.gay    master       6      True            True   
7      8          noreen.hale    murphy       6      True            True   
16    17         domingo.dyer   chelsea       7      True            True   
30    31         rosanna.reid      reid       4      True           False   
58    59  rosalinda.rodriquez    golden       6      True            True   
81    82         seymour.ball  midnight       8     False            True   
92    93    marguerite.hunter    wishes       6      True           False   
101  102            dee.mccoy    monkey       6      True            True   
115  116           emily.wise   crystal       7      True            True   
118  119     wilburn.shepherd   scanner       7      True            True   
120  121        diego.elliott    andrew       6      True            True   
130  131        zelma.rosario  replaced       8     False           False   

    password_lower common_word  
6           master        True  
7           murphy        True  
16         chelsea        True  
30            reid        True  
58          golden        True  
81        midnight        True  
92          wishes        True  
101         monkey        True  
115        crystal        True  
118        scanner        True  
120         andrew        True  
130       replaced        True

## 6. Passwords should not be your name
<p>It turns out many of our passwords were common English words too! Next up on the NIST list:</p>
<blockquote>
  <p>Verifiers SHALL compare the prospective secrets against a list that contains [...] context-specific words, such as the name of the service, the username, and derivatives thereof.</p>
</blockquote>
<p>Ok, so there are many things we could check here. One thing to notice is that our users' usernames consist of their first names and last names separated by a dot. For now, let's just flag passwords that are the same as either a user's first or last name.</p>

In [137]:
# Extracting first and last names into their own columns
#print(users['user_name'].str.split("."))
users['name_split'] = users['user_name'].str.split(".")
#users['first_name'], users['last_name'] = users['user_name'].str.extract("(^\w+).(\w+)$")
users['first_name'] = users['name_split'].str[0]
users['last_name'] = users['name_split'].str[1]

# Flagging the users with passwords that matches their names
users['uses_name_first'] = users['password'] == users.first_name
users['uses_name_last'] = users['password'] == users.last_name
#print(users.uses_name_first == True)
#print(users.uses_name_last == True)

import numpy as np
#combined_np = np.logical_or(users.uses_name_first, users.uses_name_last)
combined_np = users.uses_name_first | users.uses_name_last
#print(len(combined_np))
#print(len(users[combined_np]))
users['uses_name'] = combined_np

combined = (users.uses_name_first == True) | (users.uses_name_last == True)
print(len(combined))
#print(len(users[combined]))
#print(len(users))
#print(users)

# Counting and printing the number of users using names as passwords
# ... YOUR CODE FOR TASK 6 ...
users_name_users = users[users['uses_name']]
print(len(users_name_users))

# Taking a look at the 12 first rows
# ... YOUR CODE FOR TASK 6 ...
users_name_users.head(12)

982
50


id        user_name  password  length too_short common_password  \
11    12  milford.hubbard   hubbard       7      True           False   
22    23    jenny.woodard   woodard       7      True           False   
30    31     rosanna.reid      reid       4      True           False   
84    85  dorian.caldwell    dorian       6      True            True   
89    90    jordan.hurley    hurley       6      True            True   
112  113    juliette.rios  juliette       8     False            True   
122  123     caleb.powers     caleb       5      True           False   
136  137        rhea.ware      ware       4      True           False   
147  148      lara.bright      lara       4      True            True   
151  152  roland.whitaker    roland       6      True            True   
158  159   stefan.wheeler    stefan       6      True            True   
163  164      dixie.noble     dixie       5      True            True   

    password_lower common_word          name_split first_name last_name  \
11         hubbard       False  [milford, hubbard]    milford   hubbard   
22         woodard       False    [jenny, woodard]      jenny   woodard   
30            reid        True     [rosanna, reid]    rosanna      reid   
84          dorian       False  [dorian, caldwell]     dorian  caldwell   
89          hurley       False    [jordan, hurley]     jordan    hurley   
112       juliette       False    [juliette, rios]   juliette      rios   
122          caleb       False     [caleb, powers]      caleb    powers   
136           ware        True        [rhea, ware]       rhea      ware   
147           lara       False      [lara, bright]       lara    bright   
151         roland        True  [roland, whitaker]     roland  whitaker   
158         stefan       False   [stefan, wheeler]     stefan   wheeler   
163          dixie       False      [dixie, noble]      dixie     noble   

    uses_name_first uses_name_last uses_name  
11            False           True      True  
22            False           True      True  
30            False           True      True  
84             True          False      True  
89            False           True      True  
112            True          False      True  
122            True          False      True  
136           False           True      True  
147            True          False      True  
151            True          False      True  
158            True          False      True  
163            True          False      True

## 7. Passwords should not be repetitive
<p>Milford Hubbard (user number 12 above), what where you thinking!? Ok, so the last thing we are going to check is a bit tricky:</p>
<blockquote>
  <p>verifiers SHALL compare the prospective secrets [so that they don't contain] repetitive or sequential characters (e.g. ‘aaaaaa’, ‘1234abcd’).</p>
</blockquote>
<p>This is tricky to check because what is <em>repetitive</em> is hard to define. Is <code>11111</code> repetitive? Yes! Is <code>12345</code> repetitive? Well, kind of. Is <code>13579</code> repetitive? Maybe not..? To check for <em>repetitiveness</em> can be arbitrarily complex, but here we're only going to do something simple. We're going to flag all passwords that contain 4 or more repeated characters.</p>

In [139]:
### Flagging the users with passwords with >= 4 repeats
#print(users['password'].str.contains("(.)\1\1\1"))
# for i in range(0, 981):
#     print(users.loc[i].password)
users['too_many_repeats'] = users['password'].str.contains(r"(.)\1\1")
print(len(users[users['password_lower'].str.contains(r"(.)\1\1\1")]))

# Taking a look at the users with too many repeats
# ... YOUR CODE FOR TASK 7 ...
print(users[users['too_many_repeats']])

6
      id          user_name       password  length too_short common_password  \
4      5     araceli.wilder     acecdn3000      10     False           False   
61    62      shane.watkins      game67776       9     False           False   
111  112        jacob.boone       CAPRI333       8     False           False   
146  147        patti.dixon         555555       6      True            True   
192  193    trinidad.austin      012227257       9     False           False   
248  249    brittany.mosley       22411142       8     False           False   
294  295      gilbert.sykes       111222aa       8     False           False   
364  365         emery.ruiz         888278       6      True           False   
423  424      anastasia.fry     Dio7294666      10     False           False   
453  454  millicent.holcomb       18885169       8     False           False   
572  573   cornelia.bradley         555555       6      True            True   
593  594      gus.macdonald        455

## 8. All together now!
<p>Now we have implemented all the basic tests for bad passwords suggested by NIST Special Publication 800-63B! What's left is just to flag all bad passwords and maybe to send these users an e-mail that strongly suggests they change their password.</p>

In [141]:
# Flagging all passwords that are bad
import numpy as np
combined_np = users.too_short | users.common_password | users.common_word | users.uses_name | users.too_many_repeats
print(len(combined_np))
users['bad_password'] = combined_np

# Counting and printing the number of bad passwords
# ... YOUR CODE FOR TASK 8 ...
print(len(users[users['bad_password']]))

# Looking at the first 25 bad passwords
# ... YOUR CODE FOR TASK 8 ...
print(users[users['bad_password']].head(25))

982
438
    id         user_name    password  length too_short common_password  \
4    5    araceli.wilder  acecdn3000      10     False           False   
5    6  shawn.harrington     5278049       7      True           False   
6    7        evelyn.gay      master       6      True            True   
7    8       noreen.hale      murphy       6      True            True   
8    9       gladys.ward     lwsves2       7      True           False   
11  12   milford.hubbard     hubbard       7      True           False   
13  14     jamie.cochran      310356       6      True           False   
15  16        lorrie.gay     oZ4k0QE       7      True           False   
16  17      domingo.dyer     chelsea       7      True            True   
17  18    martin.pacheco     zvc1939       7      True           False   
18  19     shelby.massey      nickgd       6      True           False   
21  22   leticia.sanford    cocacola       8     False            True   
22  23     jenny.woodard     w

## 9. Otherwise, the password should be up to the user
<p>In this notebook, we've implemented the password checks recommended by the NIST Special Publication 800-63B. It's certainly possible to better implement these checks, for example, by using a longer list of common passwords. Also note that the NIST checks in no way guarantee that a chosen password is good, just that it's not obviously bad.</p>
<p>Apart from the checks we've implemented above the NIST is also clear with what password rules should <em>not</em> be imposed:</p>
<blockquote>
  <p>Verifiers SHOULD NOT impose other composition rules (e.g., requiring mixtures of different character types or prohibiting consecutively repeated characters) for memorized secrets. Verifiers SHOULD NOT require memorized secrets to be changed arbitrarily (e.g., periodically).</p>
</blockquote>
<p>So the next time a website or app tells you to "include both a number, symbol and an upper and lower case character in your password" you should send them a copy of <a href="https://pages.nist.gov/800-63-3/sp800-63b.html">NIST Special Publication 800-63B</a>.</p>

In [143]:
# Enter a password that passes the NIST requirements
# PLEASE DO NOT USE AN EXISTING PASSWORD HERE
new_password = "1qaz@WSX3edc$RFV"